In [6]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [7]:
import os
path= "/content/drive/My Drive/engineering/college/term 10/pattern recognition/projects/face_recognition" #Alaa
os.chdir(path)
!ls 

face_recognition.ipynb	orl_faces  Untitled0.ipynb


In [26]:

from PIL import Image
import numpy as np
import glob
import regex 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier


In [10]:



def read_dataset(path):
  data= np.zeros([400,10305])
  i=0
  iamges_list = []
  for filename in glob.glob(path+"/*/*.pgm"): 

    im=Image.open(filename)
    iamges_list.append(im)
    l=regex.split(r'/', filename)
    l2= regex.split(r's', l[1])
    arr = np.array(im)
    data[i,0:10304]=arr.reshape(1,10304)
    data[i, -1]=l2[1]
    i+=1
  return data, iamges_list


    

In [143]:
def data_splitting():
    test_X = np.zeros([200,10305])
    train_X = np.zeros([200,10305])
    test_y= np.zeros(200)
    train_y=np.zeros(200)
    for i in range(data.shape[0]):
        if i%2 == 0:
            test_X[int(i/2)]=data[i,0:10305]
            test_y[int(i/2)]= data[i,10304]
        else:
            train_X[int((i-1)/2)]=data[i,0:10305]
            train_y[int((i-1)/2)]= data[i,10304] 
    return train_X, train_y, test_X, test_y


In [0]:
def visualize(images,i):
  plt.imshow(images[i])
  

In [144]:
data, images= read_dataset("orl_faces") 
train_x, train_y, test_x, test_y= data_splitting()
#visualize(images, 32)

In [101]:
def pca():
  pca = PCA(n_components= 1000)
  new_dimensions= pca.fit_transform(train_X)
  #print(pca.explained_variance_ratio_)  
  #print(new_dimensions.shape)
  return  new_dimensions

new_dimensions= pca ()




In [146]:
# K-NN classifier
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(train_x,train_y) 
neigh.score(test_x, test_y)

0.9

In [94]:
train[1,20].shape

()

In [139]:
test_y.shape

()

In [98]:
data[1,10305]

IndexError: index 10305 is out of bounds for axis 1 with size 10305

In [138]:
train_x.shape

(200, 10305)